# Data Processing Notebook

## 1. Imports and Setup

In [1]:
import os
import pandas as pd
import numpy as np

import geopandas as gpd
import rasterio as rio

import sys
sys.path.insert(0, '../utils')
import geoutils

import logging
import warnings
logging.getLogger().setLevel(logging.ERROR)
warnings.filterwarnings("ignore")

import matplotlib.pyplot as plt
%matplotlib inline

%load_ext autoreload
%autoreload 2

## 2. File Locations

In [2]:
data_dir = "../data/"
pos_mask_dir = data_dir + 'pos_masks/'
neg_mask_dir = data_dir + 'neg_masks/'
sentinel_dir = data_dir + 'sentinel2/'

if not os.path.exists(data_dir):
    os.makedirs(data_dir)
if not os.path.exists(sentinel_dir):
    os.makedirs(sentinel_dir)
if not os.path.exists(pos_mask_dir):
    os.makedirs(pos_mask_dir)
if not os.path.exists(neg_mask_dir):
    os.makedirs(neg_mask_dir)

## 3. Download Files from GCS

In [3]:
!gsutil -q -m cp gs://immap-gee/DEFLATED_gee_*.tif {sentinel_dir}
!gsutil -q -m cp gs://immap-gee/CROPPED_gee_*.tif {sentinel_dir}
!gsutil -q -m cp gs://immap-masks/informal_settlement_masks/*.gpkg {pos_mask_dir}
!gsutil -q -m cp gs://immap-masks/negative_sample_masks/*.gpkg {neg_mask_dir}
!gsutil -q -m cp gs://immap-masks/admin_boundaries/admin_bounds.gpkg {data_dir} 
print('Operations completed.')

Operations completed.


## 4. Create Area Dictionary

In [3]:
area_dict = {area : dict() for area in ['maicao', 'riohacha', 'uribia']}
for area in area_dict:
    area_dict[area]['pos_mask_gpkg'] = '{}{}_mask.gpkg'.format(pos_mask_dir, area)
    area_dict[area]['neg_mask_gpkg'] = '{}{}-samples.gpkg'.format(neg_mask_dir, area)
    image_files, image_cropped = [], []
    for image_file in os.listdir(sentinel_dir):
        if area in image_file and 'DEFLATE' in image_file:
            image_files.append(sentinel_dir+image_file)
        if area in image_file and 'CROPPED' in image_file:
            image_cropped.append(sentinel_dir+image_file)
    area_dict[area]['images'] = sorted(image_files)
    area_dict[area]['images_cropped'] = sorted(image_cropped)
    
area_dict['maicao']

{'pos_mask_gpkg': '../data/pos_masks/maicao_mask.gpkg',
 'neg_mask_gpkg': '../data/neg_masks/maicao-samples.gpkg',
 'images': ['../data/sentinel2/DEFLATED_gee_maicao_2016.tif',
  '../data/sentinel2/DEFLATED_gee_maicao_2017.tif',
  '../data/sentinel2/DEFLATED_gee_maicao_2018.tif',
  '../data/sentinel2/DEFLATED_gee_maicao_2019.tif',
  '../data/sentinel2/DEFLATED_gee_maicao_2020.tif'],
 'images_cropped': ['../data/sentinel2/CROPPED_gee_maicao_2016.tif',
  '../data/sentinel2/CROPPED_gee_maicao_2017.tif',
  '../data/sentinel2/CROPPED_gee_maicao_2018.tif',
  '../data/sentinel2/CROPPED_gee_maicao_2019.tif',
  '../data/sentinel2/CROPPED_gee_maicao_2020.tif']}

## 5. Generate Cropped TIFF Files
<font color='red'>**Note**: No need to run the cells in this section if the CROPPED files are already downloaded!</font>

### Read Administrative Boundaries Geopackage

In [4]:
admin_bounds_file = data_dir + 'admin_bounds.gpkg'
admin_bounds = gpd.read_file(admin_bounds_file)
admin_bounds.head(2)

,admin0Name,admin0Pcod,admin1Name,admin1Pcod,admin2Name,admin2Pcod,admin2RefN,admin2AltN,admin2Al_1,date,validOn,validTo,Shape_Leng,Shape_Area,geometry
0,Colombia,CO,La Guajira,CO44,UribÃ­a,CO44847,Uribia,None,None,2009-01-01,2014-10-29,None,5.356435,0.657424,"MULTIPOLYGON (((-71.97857 11.63127, -71.97948 ..."
1,Colombia,CO,La Guajira,CO44,Manaure,CO44560,Manaure,None,None,2009-01-01,2014-10-29,None,2.034267,0.134340,"MULTIPOLYGON (((-72.26228 11.52743, -72.26231 ..."


### Generate Cropped TIFF: Masking a Raster based on a Shapefile
**Source**: https://rasterio.readthedocs.io/en/latest/topics/masking-by-shapefile.html

In [6]:
for area in area_dict:
    geometry = admin_bounds[admin_bounds.admin2RefN == area.title()]
    geometry = [geoutils.explode(geometry)['geometry'].values[0]]  
    
    images_cropped = []
    for image_file in area_dict[area]['images']: 
        output_file = image_file.replace('DEFLATED', 'CROPPED')
        with rio.open(image_file) as src:
            out_image, out_transform = rasterio.mask.mask(src, geometry, crop=True)
            out_meta = src.meta
            
        out_meta.update({
            "driver": "GTiff",
            "height": out_image.shape[1],
            "width": out_image.shape[2],
            "transform": out_transform,
            'nodata': 0
        })
        with rio.open(output_file, "w", **out_meta, compress='deflate') as dest:
            dest.write(out_image)
        images_cropped.append(output_file)
        
    area_dict[area]['images_cropped'] = sorted(images_cropped)

# Upload to GCS
for area in area_dict:
    for image_file in area_dict[area]['images_cropped']:
        !gsutil -q -m cp {image_file} gs://immap-gee/

## 6. Generate Target Raster Masks

### Positive Labels

In [6]:
for area, value in area_dict.items():
    tiff_file = value['images_cropped'][0]
    shape_file = value['pos_mask_gpkg']
    target_file = shape_file.replace('gpkg', 'tiff')
    
    geoutils.generate_mask(
         tiff_file=tiff_file, 
         shape_file=shape_file, 
         output_file=target_file, 
         plot=False
    )
    area_dict[area]['pos_mask_tiff'] = target_file

### Negative Labels

In [7]:
for area, value in area_dict.items():
    tiff_file = value['images_cropped'][0]
    shape_file = value['neg_mask_gpkg']
    target_file = shape_file.replace('gpkg', 'tiff')
    
    if os.path.isfile(shape_file):
        gdf = gpd.read_file(shape_file)
        gdf['class'] = gdf['class'].str.lower() 
        gdf = gdf[
            (gdf['class'] == 'unoccupied land') 
            | (gdf['class'] == 'formal settlement')
        ]
        shape_file = shape_file.replace('samples', 'masks')
        gdf.to_file(shape_file, driver='GPKG')
    
        geoutils.generate_mask(
             tiff_file=tiff_file, 
             shape_file=shape_file, 
             output_file=target_file, 
             plot=False
        )
    area_dict[area]['neg_mask_tiff'] = target_file

## 7. Generate Training Set

In [8]:
from tqdm import tqdm

data_all = []
for area in ['maicao', 'riohacha']: #area_dict
    print('Reading {}...'.format(area))
    
    # Read positive target mask
    pos_mask = rio.open(area_dict[area]['pos_mask_tiff'])
    pos_mask = pos_mask.read(1).ravel()
    
    # Read negative mask
    neg_mask = rio.open(area_dict[area]['neg_mask_tiff'])
    neg_mask = neg_mask.read(1).ravel()
    
    # Get sum of postive and negative mask
    mask = pos_mask + neg_mask

    data = dict()
    
    # Iterate over each year
    image_list = area_dict[area]['images_cropped']
    for image_file in tqdm(image_list, total=len(image_list)):
        year = image_file.split('_')[-1].split('.')[0]
        
        # Read each band
        raster = rio.open(image_file)
        for band_idx in range(raster.count):
            band = raster.read(band_idx+1).ravel()
            data['band{}_{}'.format(band_idx+1, year)] = band
    
    # cast to pandas dataframe
    data = pd.DataFrame(data)
    data['target'] = mask

    data = data[data.values.sum(axis=1) != 0] 
    data_all.append(data)

# Concatenate all areas
data = pd.concat(data_all)
print('Data dimensions: {}'.format(data.shape))
data.head(3)

Reading maicao...


100%|██████████| 5/5 [00:36<00:00,  7.25s/it]


Reading riohacha...


100%|██████████| 5/5 [00:57<00:00, 11.57s/it]


Data dimensions: (48242797, 61)


,band1_2016,band2_2016,band3_2016,band4_2016,band5_2016,band6_2016,band7_2016,band8_2016,band9_2016,band10_2016,...,band4_2020,band5_2020,band6_2020,band7_2020,band8_2020,band9_2020,band10_2020,band11_2020,band12_2020,target
11131,0.1492,0.1331,0.1373,0.1587,0.1735,0.2247,0.2654,0.2486,0.3075,0.0377,...,0.1442,0.1754,0.2019,0.2289,0.2352,0.2579,0.2738,0.3625,0.2616,0
16695,0.1492,0.1359,0.1543,0.1761,0.1799,0.2381,0.2838,0.2657,0.3186,0.0377,...,0.1632,0.1908,0.2187,0.2398,0.2530,0.2711,0.2738,0.3587,0.2642,0
16696,0.1492,0.1395,0.1514,0.1644,0.1799,0.2381,0.2838,0.2652,0.3186,0.0377,...,0.1498,0.1908,0.2187,0.2398,0.2464,0.2711,0.2738,0.3587,0.2642,0


In [14]:
data[data['target'] != 0].shape

(159515, 61)

In [12]:
data[data['target'] == 2].shape

(121984, 61)